In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [8]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

In [9]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [10]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [11]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [12]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [13]:
df['loc']

1     (2821 S Bayshore Dr #9a, Miami, FL 33133, USA,...
2     (Asia, 900 Brickell Key Blvd APT 2303, Miami, ...
3     (100 South Pointe Dr APT 702, Miami Beach, FL ...
4     (10295 Collins Ave, Bal Harbour, FL 33154, USA...
5     (13627 Deering Bay Dr APT 101, Coral Gables, F...
6     (20165 NE 39th Pl APT 704, Aventura, FL 33180,...
7     (3201 NE 183rd St APT 2405, Aventura, FL 33160...
8     (1250 Ocean Dr #3a, Miami Beach, FL 33139, USA...
9     (125 Jefferson Ave APT 121, Miami Beach, FL 33...
10    (400 Sunny Isles Blvd, Sunny Isles Beach, FL 3...
Name: loc, dtype: object

In [14]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [15]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Park Grove\r2821 S Bayshore Dr 9A\rMiami,1/3/2023,38,"$4,700,000.00",3233,"$1,453.76",Alina Gallart,One Sotheby's International Re,Julian Cohen,Douglas Elliman,Park Grove 2821 S Bayshore Dr 9A Miami,Park Grove,2821 S Bayshore Dr 9A Miami,"(2821 S Bayshore Dr #9a, Miami, FL 33133, USA,...","(25.7287205, -80.23808009999999, 0.0)",25.728721,-80.238080,0.0
2,Asia Condo\r900 Brickell Key Blvd 2303\rMiami,1/3/2023,700,"$3,900,000.00",3398,"$1,147.73",Joao Carvalho,"Jad Realty, LLC.",Ricardo Cruz,La Playa Properties Group Inc,Asia Condo 900 Brickell Key Blvd 2303 Miami,Asia Condo,900 Brickell Key Blvd 2303 Miami,"(Asia, 900 Brickell Key Blvd APT 2303, Miami, ...","(25.7690285, -80.1856987, 0.0)",25.769029,-80.185699,0.0
3,Continuum on South Beach\r100 S Pointe Dr 702\...,1/5/2023,57,"$2,700,000.00",1201,"$2,248.13",Sheila Rojas,Brown Harris Stevens,Timothy Allen Jr,Blackstone Intern'l Realty LLC,Continuum on South Beach 100 S Pointe Dr 702 M...,Continuum on South Beach,100 S Pointe Dr 702 Miami Beach,"(100 South Pointe Dr APT 702, Miami Beach, FL ...","(25.7676323, -80.137348, 0.0)",25.767632,-80.137348,0.0
4,Ritz-Carlton Bal Harbour\r10295 Collins Ave 91...,1/6/2023,34,"$2,450,000.00",1857,"$1,478.58",Henrik Hesselman,LoKation,Henrik Hesselman,LoKation,Ritz-Carlton Bal Harbour 10295 Collins Ave 916...,Ritz-Carlton Bal Harbour,10295 Collins Ave 916 & 917 Bal Harbour,"(10295 Collins Ave, Bal Harbour, FL 33154, USA...","(25.8989516, -80.1234047, 0.0)",25.898952,-80.123405,0.0
5,Deering Bay\r13627 Deering Bay Dr 101\rCoral G...,1/3/2023,10,"$1,900,000.00",2740,$693.43,Geraldine Brodie,Onyx Realty,Lourdes Alatriste,Douglas Elliman,Deering Bay 13627 Deering Bay Dr 101 Coral Gab...,Deering Bay,13627 Deering Bay Dr 101 Coral Gablesh,"(13627 Deering Bay Dr APT 101, Coral Gables, F...","(25.6368923, -80.2903935, 0.0)",25.636892,-80.290393,0.0
6,Porto Vita\r20165 NE 39th Pl 704\rMiami,1/4/2023,14,"$1,680,000.00",3470,$484.15,Jason Shapiro,Rising Realty of S Florida,Greg Marchese,PRESTIGE REAL ESTATE GROUP,Porto Vita 20165 NE 39th Pl 704 Miami,Porto Vita,20165 NE 39th Pl 704 Miami,"(20165 NE 39th Pl APT 704, Aventura, FL 33180,...","(25.9625045, -80.12481729999999, 0.0)",25.962505,-80.124817,0.0
7,The Peninsula Condo\r3201 NE 183 St 2405\rAven...,1/5/2023,36,"$1,657,000.00",3311,$557.91,Brian Jones,Beachfront Realty Inc,Eduard Zavulunov,Compass Florida LLC,The Peninsula Condo 3201 NE 183 St 2405 Aventura,The Peninsula Condo,3201 NE 183 St 2405 Aventura,"(3201 NE 183rd St APT 2405, Aventura, FL 33160...","(25.9456839, -80.13577169999999, 0.0)",25.945684,-80.135772,0.0
8,The Carlyle Hotel Condo\r1250 Ocean Dr 3A\rMia...,1/4/2023,33,"$1,500,000.00",2007,$747.38,Ayleen De La Vega,"Kay Luxury Properties, LLC.",Todd Nordstrom,Compass Florida LLC,The Carlyle Hotel Condo 1250 Ocean Dr 3A Miami...,The Carlyle Hotel Condo,1250 Ocean Dr 3A Miami Beach,"(1250 Ocean Dr #3a, Miami Beach, FL 33139, USA...","(25.7837906, -80.13027509999999, 0.0)",25.783791,-80.130275,0.0
9,The Courts at South Beach\r125 Jefferson Ave 1...,1/3/2023,99,"$1,220,000.00",1485,$821.55,Richard Corrales,Silverleaf Realty Group,Marco Druzetich,Douglas Elliman,The Courts at South Beach 125 Jefferson Ave 12...,The Courts at South Beach,125 Jefferson Ave 121 Miami Beach,"(125 Jefferson Ave APT 121, Miami Beach, FL 33...","(25.7705751, -80.13698819999999, 0.0)",25.770575,-80.136988,0.0
10,0400 Sunny Isles\r400 Sunny Isles Blvd 1016\rS...,1/4/2023,51,"$1,020,000.00",1553,$656.79,Karin Hartman,Hartman Realty Group,Gladys Martinez,Elite Properties & Investments,0400 Sunny Isles 400 Sunny Isles Blvd 1016 Sun...,,0400 Sunny Isles 400 Sunny Isles Blvd 1016 Sun...,"(400 Sunny Isles Blvd, Sunny Isles Beach, FL 3...","(25.9291821, -80.12832829999999, 0.0)",25.929182,-80.128328,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [16]:
df.at[10,'building_name']=('400 Sunny Isles')
df.at[10,'address_only']=('400 Sunny Isles Blvd')

In [17]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [18]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 1st - January 7th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [14]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [15]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_123122
